# Chapter 10

This chapter is going to introduce three complex types of data that are extremely important for your smart contracts.  
So far, the smart contracts we have written were quite simple: they received data as input, applied some logic to the data and output a result. However, this is not what most contracts do. Smart contracts are an interesting solution to many problems because they can retain information and the new input sent to the contract can be compared and/or used against the data that's already in the contract!

In a previous chapter, we introduced the `list` type. Lists allow us to store multiple values of the same type in an ordered fashion. However, they lack some features that you may need for your smart contract. For example, it is not possible natively to know if a list contains a certain value (i.e without implementing a loop).

Michelson provides three other types of values that can store other values: sets, maps and big maps. Their different properties will be explained below but in a nutshell, sets are lists of unique elements, maps are tables where each value matches a unique key and big maps are maps containing a large number of values. Let's start with sets!

## Working with sets

Sets are a sorted collection of unique values. A `set` is like a `list` with the main difference of storing only unique values. While `{ 1 ; 2 ; 1 ; 2 ; 1 ; 2 }` is a valid value of type `(list int)`, it wouldn't be a valid value of type `(set int)`. Here is how to create an empty set:

In [2]:
storage (set int) ;
parameter unit ;
code {
    DROP ;
    EMPTY_SET int ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit { 1 };

storage (set int);
parameter unit;
code { DROP ; EMPTY_SET int ; NIL operation ; PAIR };
RUN: use %default; drop all; push (Unit, {1});
DROP: pop (Unit, {1});
EMPTY_SET: push [];
NIL: push [];
PAIR: pop [], []; push ([], set());

value,type
{},set int


You can use the **`EMPTY_SET`** instruction to create an empty set. It takes 1 argument, the type of the elements you will store in the set. It must be a comparable type: you can, for example, put strings and integers in a set but you cannot put a map or a set inside a set.

Next, you want to save some data inside the set. You can use the **`UPDATE`** instruction to push new values in the set. This instruction requires a little setup illustrated in the example below:

In [3]:
storage (set int) ;
parameter unit ;
code {
    DROP ;
    EMPTY_SET int ;
    PUSH bool True;
    PUSH int 5 ;
    UPDATE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit { 1 };

storage (set int);
parameter unit;
code { DROP ; EMPTY_SET int ; PUSH bool True ; PUSH int 5 ; UPDATE ; NIL operation ; PAIR };
RUN: use %default; drop all; push (Unit, {1});
DROP: pop (Unit, {1});
EMPTY_SET: push [];
PUSH: push True;
PUSH: push 5;
UPDATE: pop 5, True, []; push {5};
NIL: push [];
PAIR: pop [], {5}; push ([], {5});

value,type
{ 5 },set int


We could also have used the set in the storage:

In [4]:
storage (set int) ;
parameter unit ;
code {
    CDR ;
    PUSH bool True;
    PUSH int 5 ;
    UPDATE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit { 1 };

storage (set int);
parameter unit;
code { CDR ; PUSH bool True ; PUSH int 5 ; UPDATE ; NIL operation ; PAIR };
RUN: use %default; drop all; push (Unit, {1});
CDR: pop (Unit, {1}); push {1};
PUSH: push True;
PUSH: push 5;
UPDATE: pop 5, True, {1}; push {1, 5};
NIL: push [];
PAIR: pop [], {1, 5}; push ([], {1, 5});

value,type
{ 5 ; 1 },set int


Here is what happens in the stack: the **`UPDATE`** instruction requires 3 elements in the stack before proceeding: 
1. The element to add to the set
2. A boolean value set to `True` to indicate that you want to add a new element to the set
3. The set in which you want to store the new value
If the element is already present in the stack, the execution will just continue without any change. Otherwise, the new element will be pushed at the head position of the set.

If you set the boolean value to `False` and provide a value that already exists in the set, **`UPDATE`** will remove this element from the set:

In [6]:
storage (set int) ;
parameter unit ;
code {
    CDR ;
    PUSH bool False;
    PUSH int 5 ;
    UPDATE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit { 1 ; 2 ; 3 ; 4 ; 5 };

storage (set int);
parameter unit;
code { CDR ; PUSH bool False ; PUSH int 5 ; UPDATE ; NIL operation ; PAIR };
RUN: use %default; drop all; push (Unit, {1, 2, 3, 4, 5});
CDR: pop (Unit, {1, 2, 3, 4, 5}); push {1, 2, 3, 4, 5};
PUSH: push False;
PUSH: push 5;
UPDATE: pop 5, False, {1, 2, 3, 4, 5}; push {1, 2, 3, 4};
NIL: push [];
PAIR: pop [], {1, 2, 3, 4}; push ([], {1, 2, 3, 4});

value,type
{ 1 ; 2 ; 3 ; 4 },set int


Now you understand why you must set up correctly the stack before using **`UPDATE`**: the 3 required elements must be in the right order and the boolean element must be set to the right value in order to add or remove a value from the set. Here is a more complex example:

In [8]:
storage (set string) ;
parameter (pair string string) ;
code {
    UNPPAIAIR ;
    DIP { SWAP } ;
    PUSH bool True ;
    SWAP ;
    UPDATE ;
    SWAP ;
    PUSH bool False ;
    SWAP ;
    UPDATE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default (Pair "mango" "apple") { "banana" ; "apple" ; "strawberry" };

storage (set string);
parameter (pair string string);
code { { { DUP ; CAR ; DIP { CDR } } ; { DUP ; CAR ; DIP { CDR } } } ; DIP { SWAP } ; PUSH bool True ; SWAP ; UPDATE ; SWAP ; PUSH bool False ; SWAP ; UPDATE ; NIL operation ; PAIR };
RUN: use %default; drop all; push (('mango', 'apple'), {'banana', 'strawberry', 'apple'});
DUP: push (('mango', 'apple'), {'banana', 'strawberry', 'apple'});
CAR: pop (('mango', 'apple'), {'banana', 'strawberry', 'apple'}); push ('mango', 'apple');
DIP: protect 1 item(s);
CDR: pop (('mango', 'apple'), {'banana', 'strawberry', 'apple'}); push {'banana', 'strawberry', 'apple'};
restore 1 item(s);
DUP: push ('mango', 'apple');
CAR: pop ('mango', 'apple'); push mango;
DIP: protect 1 item(s);
CDR: pop ('mango', 'apple'); push apple;
restore 1 item(s);
DIP: protect 1 item(s);
SWAP: pop apple, {'banana', 'strawberry', 'apple'}; push apple; push {'banana', 'strawberry', 'apple'};
restore 1 item(s);
PUSH: push True;
SWAP: pop True, mango; push True; push mango;

value,type
"{ ""mango"" ; ""banana"" ; ""strawberry"" }",set string


First, we deconstruct the parameter with **`UNPPAIAIR`** to get our two fruits on the stack with the set of fruits below. Then, we push *mango* into the set of fruits before removing *apple*. A few **`SWAP`** instructions are necessary to put all the elements in the right order. Observe how the boolean values are set to `True` and `False` according to the effect you want to create on the set.

Before trying to remove or add a value to a set, it would be great to check if the value is already inside. This is what you can achieve with the **`MEM`** instruction:

In [11]:
storage (set string) ;
parameter string ;
code {
    UNPAIR ;
    DIP { DUP } ;
    DUP ;
    DIP { SWAP } ;
    MEM ;
    IF { FAIL } { PUSH bool True ; SWAP ; UPDATE } ;
    NIL operation ;
    PAIR ;
} ;

RUN %default "mango" { "banana" ; "apple" ; "strawberry" };

storage (set string);
parameter string;
code { { DUP ; CAR ; DIP { CDR } } ; DIP { DUP } ; DUP ; DIP { SWAP } ; MEM ; IF { { UNIT ; FAILWITH } } { PUSH bool True ; SWAP ; UPDATE } ; NIL operation ; PAIR };
RUN: use %default; drop all; push ('mango', {'banana', 'strawberry', 'apple'});
DUP: push ('mango', {'banana', 'strawberry', 'apple'});
CAR: pop ('mango', {'banana', 'strawberry', 'apple'}); push mango;
DIP: protect 1 item(s);
CDR: pop ('mango', {'banana', 'strawberry', 'apple'}); push {'banana', 'strawberry', 'apple'};
restore 1 item(s);
DIP: protect 1 item(s);
DUP: push {'banana', 'strawberry', 'apple'};
restore 1 item(s);
DUP: push mango;
DIP: protect 1 item(s);
SWAP: pop mango, {'banana', 'strawberry', 'apple'}; push mango; push {'banana', 'strawberry', 'apple'};
restore 1 item(s);
MEM: pop mango, {'banana', 'strawberry', 'apple'}; push False;
IF: pop False;
PUSH: push True;
SWAP: pop True, mango; push True; push mango;
UPDATE: pop mango, True, {'banana', 'strawberry', 'apple'}; 

value,type
"{ ""mango"" ; ""banana"" ; ""apple"" ; ""strawberry"" }",set string


In this contract, we duplicate the values passed in the parameter because **`MEM`** is going to remove the element and the set we want to test. If the value is not in the set, it will be added, otherwise, the contract will fail. As expected, *mango* is not in the set, so the value gets added to the final set. If you replace *mango* with *apple*, you will see the contract fail.

As it is also the case for other types storing multiple values, you can check the size of a set by using the **`SIZE`** instruction:

In [14]:
storage nat ;
parameter (set string) ;
code {
    CAR ;
    SIZE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default { "banana" ; "apple" ; "strawberry" } 0;

storage nat;
parameter (set string);
code { CAR ; SIZE ; NIL operation ; PAIR };
RUN: use %default; drop all; push ({'banana', 'strawberry', 'apple'}, 0);
CAR: pop ({'banana', 'strawberry', 'apple'}, 0); push {'banana', 'strawberry', 'apple'};
SIZE: pop {'banana', 'strawberry', 'apple'}; push 3;
NIL: push [];
PAIR: pop [], 3; push ([], 3);

value,type
3,nat


The **`SIZE`** instruction returns a value of type `nat`. Add more strings to the set to see it working!

The last available instruction for sets is **`ITER`**. **`ITER`** allows you to loop through a set and run some code at each iteration. Let's see it in action!

In [25]:
storage (set int) ;
parameter unit ;
code {
    CDR ;
    EMPTY_SET int ;
    SWAP ;
    ITER { PUSH int 3 ; ADD ; PUSH bool True ; SWAP ; UPDATE } ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit { 1 ; 2 ; 3 };

storage (set int);
parameter unit;
code { CDR ; EMPTY_SET int ; SWAP ; ITER { PUSH int 3 ; ADD ; PUSH bool True ; SWAP ; UPDATE } ; NIL operation ; PAIR };
RUN: use %default; drop all; push (Unit, {1, 2, 3});
CDR: pop (Unit, {1, 2, 3}); push {1, 2, 3};
EMPTY_SET: push [];
SWAP: pop [], {1, 2, 3}; push []; push {1, 2, 3};
ITER: pop {1, 2, 3}; push 1;
PUSH: push 3;
ADD: pop 3, 1; push 4;
PUSH: push True;
SWAP: pop True, 4; push True; push 4;
UPDATE: pop 4, True, []; push {4};
push 2;
PUSH: push 3;
ADD: pop 3, 2; push 5;
PUSH: push True;
SWAP: pop True, 5; push True; push 5;
UPDATE: pop 5, True, {4}; push {4, 5};
push 3;
PUSH: push 3;
ADD: pop 3, 3; push 6;
PUSH: push True;
SWAP: pop True, 6; push True; push 6;
UPDATE: pop 6, True, {4, 5}; push {4, 5, 6};
NIL: push [];
PAIR: pop [], {4, 5, 6}; push ([], {4, 5, 6});

value,type
{ 6 ; 5 ; 4 },set int
